In [16]:
#IMPORT NECESSARY PACKAGES
import twitter
import json

In [17]:
#GET ALL TWITTER API KEYS
CONSUMER_KEY = '6IE30Alw097TCFuEWCC9PYftF'
CONSUMER_SECRET = 'Qnp2OSsXxHFujmTwTFPioHHDfRadNpFiKOSZILhoHyh18wqmho'
OAUTH_TOKEN = '518127850-X9SBhwb1ovkUatDL9r0IWS3UbaOUOGHNovWLN1fo'
OAUTH_TOKEN_SECRET = '7EYclH86kvNaghe5DfiJaNo5qe1JjP7nsixwcZ8nylndJ'
auth = twitter.oauth.OAuth(OAUTH_TOKEN,OAUTH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

In [18]:
#GET KAMRAN SHAIKH INFO
auth = twitter.oauth.OAuth(OAUTH_TOKEN,OAUTH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

screen_name = "KAMRANSHAIKH6"
user_id = "518127850"
response = twitter_api.users.show(screen_name=screen_name)
print(json.dumps(response,sort_keys=True,indent=1))

URLError: <urlopen error [Errno 51] Network is unreachable>

In [ ]:
import sys
import time
from urllib.request import URLError
from http.client import BadStatusLine
import twitter

#MAKE TWITTER REQUEST
def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw): 
    
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
    
        if wait_period > 3600: # Seconds
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
    
        # See https://dev.twitter.com/docs/error-codes-responses for common codes
    
        if e.e.code == 401:
            print('Encountered 401 Error (Not Authorized)', file=sys.stderr)
            return None
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        elif e.e.code == 429: 
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print('Retrying in 15 minutes...ZzZ...', file=sys.stderr.flush())
                time.sleep(60*15 + 5)
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print >> sys.stderr, 'Encountered %i Error. Retrying in %i seconds' % \
                (e.e.code, wait_period)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function
    
    wait_period = 2 
    error_count = 0 

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError as e:
            error_count = 0 
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise


In [ ]:
response = make_twitter_request(twitter_api.users.lookup, 
                                screen_name=screen_name)
print(json.dumps(response, indent=1))

In [ ]:
from functools import partial
from sys import maxsize

#GET ALL FRIENDS AND FOLLOWERS USER
def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=maxsize, followers_limit=maxsize): #GET SCREEN_NAME OR ID 
    
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None),     "Must have screen_name or user_id, but not both"
    
    # See https://dev.twitter.com/docs/api/1.1/get/friends/ids and
    # https://dev.twitter.com/docs/api/1.1/get/followers/ids for details
    # on API parameters
    
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids, 
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids, 
                                count=5000)

    friends_ids, followers_ids = [], []
    
    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"], 
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:
        
        if limit == 0: continue
        
        cursor = -1
        while cursor != 0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
            #print >> sys.stderr, 'Fetched {0} total {1} ids for {2}'.format(len(ids), 
             #                                       label, (user_id or screen_name))
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances
        
            if len(ids) >= limit or response is None:
                break

    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       user_id=screen_name, 
                                                       friends_limit=500, 
                                                       followers_limit=500)
print(friends_ids)
print(followers_ids)

In [ ]:
def get_user_profile(twitter_api, screen_names=None, user_ids=None):
   
    # Must have either screen_name or user_id (logical xor)
    assert (screen_names != None) != (user_ids != None), \
    "Must have screen_names or user_ids, but not both"
    
    items_to_info = {}

    items = screen_names or user_ids
    
    while len(items) > 0:

        # Process 100 items at a time per the API specifications for /users/lookup.
        # See https://dev.twitter.com/docs/api/1.1/get/users/lookup for details.
        
        items_str = ','.join([str(item) for item in items[:100]])
        items = items[100:]

        if screen_names:
            response = make_twitter_request(twitter_api.users.lookup, 
                                            screen_name=items_str)
        else: # user_ids
            response = make_twitter_request(twitter_api.users.lookup, 
                                            user_id=items_str)
    
        for user_info in response:
            if screen_names:
                items_to_info[user_info['screen_name']] = user_info
            else: # user_ids
                items_to_info[user_info['id']] = user_info

    return items_to_info
get_friends_followers_ids(twitter_api, user_id=user_id,
                              friends_limit=maxsize, followers_limit=maxsize)

In [ ]:
def get_top_5(twitter_api , user_id):
    id = user_id
    friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       user_id=id, 
                                                       friends_limit=500, 
                                                       followers_limit=500)
    r_friends = set(friends_ids) & set(followers_ids) #GET SET WHO ARE BOTH FRIENDS AND FOLLOWERS
    top_5_list = [] #List of top 5 followers
    top_5_dict = {} #Dictionary style {id:follower_count}
    user_info = {} #Collect user info based on user_id
    followers_count = {} #Collect followers_count info based on user_id   
    
    for x in r_friends: #Iterate through reciprocal friends
        json_data = json.dumps(get_user_profile(twitter_api, user_ids=[x]), indent = 1)
        user_info = json.loads(json_data)
        followers_count[str(x)] = (user_info[str(x)]['followers_count']) 
    
    length_top5 = 0
    if len(followers_count) < 5:
        length_top5 = len(followers_count)
    else:
        length_top5 = 5

    for x in range(length_top5):
        popular_followers = max(followers_count.keys(), key=(lambda k: followers_count[k]))
        top_5_dict[popular_followers] = followers_count[popular_followers]
        top_5_list.append(int(popular_followers)) 
        del followers_count[popular_followers] 
    return top_5_list
get_top_5(twitter_api,user_id)

In [ ]:
starting_id = "518127850"
graph = []
response = get_top_5(twitter_api, user_id)  
ids = next_queue = response

print("Crawling through followers")

level = 1
print("Level 1: ")
print("User ID: ", starting_id ,",Top 5 Friends are: ", response, "\n")
max_level = 3         #MAX DEPTH OF GRAPH
while level < max_level:
    level += 1
    print("Level: ", level)
    (queue, next_queue) = (next_queue, [])
    for id in queue:
        graph_d = {}
        response = get_top_5(twitter_api, user_id=id)  
        next_queue += response
        print("User ID: ", id , ",Top 5 Friends are: ", response, "\n") #PRINT USER_ID AND HIS/HER TOP5 FOLLOWERS
        graph_d[id] = response #ADD TOP5 IN GRAPH DICTIONARY
        graph.append(graph_d) #ADD TOP 5 IN GRAPH
        
    ids += next_queue

In [ ]:
first_user = {}
response = get_top_5(twitter_api, user_id)
first_user[starting_id] = response
graph.insert(0, first_user)

import networkx as nx
import matplotlib.pyplot as plt



node_graph = nx.MultiGraph()
for g in graph:    #Iterate through graph
    for k,v in g.items(): #Iterate through values in graph
        for y in v:                
            node_graph.add_edge(k,y)    #Add key and id to graph      



nx.draw(node_graph, with_labels=True) #Draw graph
plt.savefig("node_graph.png") #Save graph as file node_graph
plt.show() #Show graph

print("Diameter: ", nx.diameter(node_graph)) #Print out diameter
dist = nx.average_shortest_path_length(node_graph) #Get average distance by taking average of shorted path length
print("Average distance: ", dist)
